# LA County coronavirus vaccine administration by city/community

### Load Python tools

In [1]:
%load_ext lab_black

In [2]:
import pandas as pd
import geopandas as gpd
import altair as alt
import altair_latimes as lat

In [3]:
alt.themes.register("latimes", lat.theme)
alt.themes.enable("latimes")
pd.options.display.max_columns = 50
pd.options.display.max_rows = 1000
alt.data_transformers.disable_max_rows()

DataTransformerRegistry.enable('default')

### Get the county-level data from CDC

In [19]:
# Pulled from here: http://www.publichealth.lacounty.gov/media/coronavirus/locations.htm

In [20]:
url = "http://www.publichealth.lacounty.gov/media/coronavirus/locations.htm#vaccinated"

In [21]:
tables = pd.read_html(url)

In [22]:
df = tables[5]

In [27]:
df.columns = (
    df.columns.str.strip()
    .str.lower()
    .str.replace(" ", "_")
    .str.replace("(", "")
    .str.replace("*", "")
    .str.replace("-", "_")
)

---

### Export to csv

In [28]:
df.to_csv("output/lacounty_vaccinations_by_city_community.csv", index=False)

In [29]:
df.head()

,city/community,number_of_persons_vaccinated,population_2019,percentage_of_people_vaccinated
0,City of Agoura Hills,6789,17447,38.9
1,City of Alhambra,26633,71313,37.3
2,City of Arcadia,18704,49666,37.7
3,City of Artesia,4895,13877,35.3
4,City of Avalon,308,3000,10.3
